In [1]:
ID_GPU = 1  # set active GPU here (if no GPU is available, this has no effect)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(ID_GPU)

from keras import backend as K
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.
/home/antoine/miniconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
import pandas as pd
from dateutil import parser
from pathlib import Path

import pickle
import numpy as np

from policy.battery import Battery
from policy.simulation import Simulation
from policy.policy import Policy

from policy.constants import NUM_THREADS

In [4]:
# list of site ids
site_id_list = [int(name.split('.')[0]) for name in os.listdir('../data/train')]
site_id_list = sorted(site_id_list)
print(site_id_list)

[1, 2, 3, 4, 7, 10, 11, 12, 29, 31, 32]


In [5]:
for site_id in [32]:
    print("                     ")
    print("---------------------")
    print("site", site_id)
    print("---------------------")
    print("                     ")
    
    ## Fix random seed
    np.random.seed(1)
    tf.set_random_seed(2)

    simulation_dir = (Path(os.getcwd())/os.pardir).resolve()
    data_dir = simulation_dir/'data'

    # Load data from battery
    metadata_path = data_dir/'metadata.csv'
    metadata = pd.read_csv(metadata_path, index_col=0)
    battery_parameters = metadata.loc[site_id]

    batt = {}
    for batt_id in [1, 2]:
        batt[batt_id] = Battery(capacity=battery_parameters[f"Battery_{batt_id}_Capacity"] * 1000,
                                charging_power_limit=battery_parameters[f"Battery_{batt_id}_Power"] * 1000,
                                discharging_power_limit=-battery_parameters[f"Battery_{batt_id}_Power"] * 1000,
                                charging_efficiency=battery_parameters[f"Battery_{batt_id}_Charge_Efficiency"],
                                discharging_efficiency=battery_parameters[f"Battery_{batt_id}_Discharge_Efficiency"])

    # Load data from site
    df = {}
    for type_simu in ["train", "submit"]:
        site_data_path = data_dir/f"{type_simu}"/f"{site_id}.csv"

        df[type_simu] = pd.read_csv(site_data_path)
        df[type_simu].index = [parser.parse(date_str) for date_str in df[type_simu]['timestamp']]

    # factors for normalizing inputs
    norm = {}
    norm["battery"] = batt[2].capacity
    norm["load"] = np.max(df["train"].actual_consumption)
    norm["pv"] = np.max(df["train"].actual_pv)
    norm["price"] = np.max(df["train"].price_buy_00)

    # simu (used for precomputations)
    simu = {}
    for type_simu in ["train", "submit"]:
        simu[type_simu] = Simulation(df[type_simu], norm, num_threads=NUM_THREADS)
        simu[type_simu].precompute()
    
    # save norm in pickle file (to be used at deploy time)
    pickle_path = f'../output/policy/norms/norm_s{site_id}.p'
    with open(pickle_path, 'wb') as f:
        pickle.dump(norm, f)
    
    # train policy
    p = Policy(simu, batt, norm)
    p.train()
    
    # save deploy model
    p.save(site_id)
    
    # clear everything
    K.clear_session()
    df = {}
    simu = {}

                     
---------------------
site 32
---------------------
                     
Epoch 1/1
390/390 [==============================] - 903s 2s/step - loss: -0.1960 - score_loss: -0.1960 - score_fair_loss: -0.1754 - sigmoid_loss: 0.4699 - efficiency_loss: 0.9500 - val_loss: -0.1627 - val_score_loss: -0.1627 - val_score_fair_loss: -0.1627 - val_sigmoid_loss: 0.4490 - val_efficiency_loss: 0.9500
Epoch 2/2
390/390 [==============================] - 802s 2s/step - loss: -0.2175 - score_loss: -0.2175 - score_fair_loss: -0.1893 - sigmoid_loss: 0.4562 - efficiency_loss: 0.9500 - val_loss: -0.1787 - val_score_loss: -0.1787 - val_score_fair_loss: -0.1787 - val_sigmoid_loss: 0.4350 - val_efficiency_loss: 0.9500
Epoch 3/3
390/390 [==============================] - 781s 2s/step - loss: -0.2228 - score_loss: -0.2228 - score_fair_loss: -0.1999 - sigmoid_loss: 0.4477 - efficiency_loss: 0.9500 - val_loss: -0.1850 - val_score_loss: -0.1850 - val_score_fair_loss: -0.1850 - val_sigmoid_loss:

KeyboardInterrupt: 